## Problem
### Using the available information, we need you to identify clusters of accommodation that bring the most positive impact to the community, allowing a wider range of actors to participate in travel and tourism as consumers and/or providers.

The main dataset is yet to be released by the contest organizers. Meanwhile we can look at the secondary data. We assume that this data is collected to find out the factors that are considered to bring most positive impact on the society.   

<span style="color:red"><b>We(Vishal/Shagun)</b> can simultaneously work on this notebook, under headings assigned to us.</span>

<b> Note: 
    * Before pushing the code to the repo, always remember to clear the output first. Cell -> All Output -> clear.
    * Document the code really well. This work style of working on same repo will help each other a lot if it is very well documented
    * Always mention the exact data source giving the the url so that the other person can download it as we are not going to push the data to the repo. 
</b>

## Official Data 
Main data of the problem is stored in the a tsv file called data.tsv. Let's read the data.

In [ ]:
import pandas as pd 

mainData = pd.read_csv("../../data/raw/official/data.tsv", sep='\t')

mainData.head()

This looks about right. Let's do the data profiling to get an insight of the data.

In [ ]:
import pandas_profiling

profile = mainData.profile_report(title='mainData Profiling Report')
profile.to_file(output_file="../../data/processed/mainData.html")

As a profiling report could be very big, we can write the report in a html file and save in data/processed folder. Also, doing the same for other datasets. 

In [ ]:
# profiling counties data
countiesData = pd.read_csv("../../data/raw/official/counties.tsv", sep='\t')
profile = countiesData.profile_report(title='countiesData Profiling Report')
profile.to_file(output_file="../../data/processed/countiesData.html")

In [ ]:
# profiling  population characteristics for US counties for a period of 2010 to 2018 data
censusData = pd.read_csv("../../data/raw/official/cc-est2018-alldata/cc-est2018-alldata.csv", encoding="ISO-8859-1")
profile = censusData.profile_report(title='censusData Profiling Report')
profile.to_file(output_file="../../data/processed/censusData.html")

The <b>air_outbound_popularity_bucket</b> is highly correlated with <b>air_inbound_popularity_bucket <span style="color:green">(ρ = 0.9962258185)</span></b>. So we can drop air_outbound_popularity from the table.  

In [ ]:
mainData = mainData.drop(['air_outbound_popularity_bucket'], axis=1)
mainData.head()

<b>countyfp</b> is the funny one. Although, the total number of counties in USA and each state matches the count but the values are assigned quite randomly.   

In [ ]:
import numpy as np

# Number of counties in 3 states by alphabatical order
print(mainData.groupby('state_code')['countyfp'].nunique()[:3], '\n')
# Total number of counties in USA
print(mainData.groupby('state_code')['countyfp'].nunique().sum(), '\n')
# values of countyfp for randomly selected 5 states.
print(np.sort(mainData[mainData.state_code == 'VA'].countyfp.unique()))
print(np.sort(mainData[mainData.state_code == 'AK'].countyfp.unique()))
print(np.sort(mainData[mainData.state_code == 'RI'].countyfp.unique()))
print(np.sort(mainData[mainData.state_code == 'TX'].countyfp.unique()))


So we have repeating values for the countyfp for each state. But this is not the case with <b>geoid</b>. <b>gioid</b> has unique values for all the counties ranging from 1001 to 56037. Also, similar to countyfp there is no obvious pattern in assigning the values.  

In [ ]:
# values of geoid for randomly selected 2 states.
print(np.sort(mainData[mainData.state_code == 'AK'].geoid.unique()))
print(np.sort(mainData[mainData.state_code == 'AL'].geoid.unique()))

<b>lodging_num_reviews_bucket</b> is highly correlated to <b>lodging_inventory_bucket <span style="color:green">(ρ = 0.929536127)</span></b> and <b>lodging_popularity_bucket</b> is highly correlated to <b>lodging_num_reviews_bucket<span style="color:green">(ρ = 0.9485770051)</span></b> . So we can drop both of these fields. 

In [ ]:
mainData = mainData.drop(['lodging_num_reviews_bucket','lodging_popularity_bucket'], axis=1)
mainData.head()

<b>state_code</b> and <b>statefp</b> are basically same thing. We can drop state_code as well for Analysis.  

In [ ]:
mainData = mainData.drop(['state_code'], axis=1)
mainData.head()

### Fixing the missing values. 

<b>lodging_avg_review_rating, lodging_avg_star_rating</b> and <b>lodging_inventory_bucket</b> has 59.3%, 64.1% and 51.1% of the values missing.

In [ ]:
# Checking the summary of the lodging_avg_review_rating for Vacation rental true and false
nonVacationRentalReview = mainData[mainData.is_vacation_rental == 0].lodging_avg_review_rating
vacationRentalReview = mainData[mainData.is_vacation_rental == 1].lodging_avg_review_rating
print("Non Vacational Rental Review Summary:", nonVacationRentalReview.describe())
print("\n Vacational Rental Review Summary:", vacationRentalReview.describe())

For <b>lodging_avg_review_rating</b> the distribution is very Gaussian like and low standard deviation from the mean. We see a difference in mean and standard deviation of review rating. So we can replace Nan for both values differently.   

In [ ]:
# filling Nan with mean for vacation and non vacation rental and then replacing the original column
x = mainData.loc[mainData.is_vacation_rental == 0]['lodging_avg_review_rating'].fillna(3.9)
x = pd.DataFrame({'lodging_avg_review_rating' : x})
y = mainData.loc[mainData.is_vacation_rental == 1]['lodging_avg_review_rating'].fillna(4.5)
y = pd.DataFrame({'lodging_avg_review_rating' : y})
frames = [x, y]
z = pd.concat(frames)
# replacing the original lodging_avg_review_rating
mainData['lodging_avg_review_rating'] = z.sort_index()

We need to do more for the left two fields than just replacing the value with mean. We can come back to them if these value will be needed. 

### Feature Engineering
Let's start building the features and final dataset that will be used for clustering. lets build the data county wise. 

In [ ]:
# moving geoid to final dataset
finalData = pd.DataFrame({'geoid' : np.sort(mainData.geoid.unique())})
finalData.head()

### Customer Satisfaction
Customer satisfaction is the first feature we are going to add to the dataset. We can take <b>lodging_avg_review_rating, lodging_avg_star_rating</b> take as customer satisfaction. We have already filler the missing values in the lodging_avg_review_rating, so we can add that to the data set straight up. But the rating is given for different years in the main Dataset. Lets take the latest value and average the value for vacation rental and non vacation rental.  

In [ ]:
from tqdm import tqdm

# initialize the value in new column with float value
finalData['CustomerSatisfactionAvgReviewRating'] = 0.0

# taking average of latest avg review rating for vacation rental and non vacation rental
for index, value in tqdm(finalData['geoid'].items()):
    finalData.CustomerSatisfactionAvgReviewRating[index] = mainData[mainData.geoid == finalData.loc[index]['geoid']][-2:]['lodging_avg_review_rating'].values.mean()
    
finalData.head()

The regenerating the missing values of the <b>lodging_avg_star_rating</b> will be little trickier. <span style="color:red">We need to apply a Machine Learning Algorithm to regenerate the value.</span>

We are not interested in all the values, but in the latest ones. Let's start with creating a new dataframe with lesser values. Also, check if value is not NaN. 

In [ ]:
# taking oly the latest values from the dataset
StarData = mainData[(mainData.months == '2018-10,2018-11') & (mainData['lodging_avg_star_rating'].notnull())]
StarData.head()

Some features are not useful for machine learning, we can remove them from the dataFrame.  

In [ ]:
averageStarData = StarData.drop(['statefp','countyfp','geoid','months'], axis=1)
averageStarData.head()

Our data is all set to be worked on. 

In [ ]:
# splitting train and test data
from sklearn.model_selection import train_test_split
train , test = train_test_split(averageStarData, test_size = 0.3)

x_train = train.drop('lodging_avg_star_rating', axis=1)
y_train = train['lodging_avg_star_rating']

x_test = test.drop('lodging_avg_star_rating', axis = 1)
y_test = test['lodging_avg_star_rating']

In [ ]:
# Scaling the features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled)

x_test_scaled = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled)

In [ ]:
# taking a look at the error rate for different k values
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

rmse_val = [] #to store rmse values for different k
for K in range(30):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(x_train, y_train)  #fit the model
    pred=model.predict(x_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
#plotting the rmse values against k values
curve = pd.DataFrame(rmse_val) #elbow curve 
curve.plot()

The RMSE value decreases as we increase the k value. At <b>k= 23</b>, the <b>RMSE is approximately 0.2859</b>, and shoots up on further increasing the k value. We can safely say that k=23 will give us the best result in this case.

In [ ]:
model = neighbors.KNeighborsRegressor(n_neighbors = 23)
model.fit(x_train, y_train)  #fit the model

In [ ]:
nullStarData = mainData[(mainData.months == '2018-10,2018-11') & (mainData['lodging_avg_star_rating'].isnull())]
nullStarData = nullStarData.reset_index(drop=True)
nullaverageStarData = nullStarData.drop(['statefp','countyfp','geoid','months', 'lodging_avg_star_rating'], axis=1)

# taking the mean of the missing values for the lodging_inventory_bucket and lodging_price_bucket 
mean1 = nullaverageStarData['lodging_inventory_bucket'].mean() 
mean2 = nullaverageStarData['lodging_price_bucket'].mean()

nullaverageStarData['lodging_inventory_bucket'].fillna(mean1, inplace =True)
nullaverageStarData['lodging_price_bucket'].fillna(mean2, inplace =True)

test_scaled = scaler.fit_transform(nullaverageStarData)
test = pd.DataFrame(test_scaled)
#predicting on the test set and creating submission file
predict = model.predict(test)

In [ ]:
# initialize the value in new column with float value
finalData['CustomerSatisfactionAvgStarRating'] = 0.0

# Adding the values of prediction from nullStarData and original data of not null values from StarData
for index, value in tqdm(finalData['geoid'].items()):
    if(value in nullStarData['geoid'].unique()): # value present in nullStarData
        geoidsIndex = nullStarData.index[nullStarData['geoid'] == value].values # get all the indexes
        for geoindex in geoidsIndex:
            finalData.CustomerSatisfactionAvgStarRating[index] = predict[geoindex]
    if(value in StarData['geoid'].unique()):
        starRatings = StarData[StarData['geoid'] == value]['lodging_avg_star_rating'].values # get all the values
        for starRating in starRatings:
            finalData.CustomerSatisfactionAvgStarRating[index] = starRating

In [ ]:
finalData.head()

With this our second feature is also ready :-) Let's save the dataframe locally to pick from here when we restart the kernel. 

In [ ]:
finalData.to_csv(r'../../data/processed/tempDataFrameLandmark/finalDataCutomerSatisfaction.csv')

We are interested in finding out which counties have more number of tourist coming. Also, number of tourist per population is more important than number of tourists itself.     

One of the field that give a good insight into this is <b>lodging_inventory_bucket</b>. Showing a bucket of number of properties. Let's create an indicator by dividing this value by the population of the county. So, lets first add the population of county in finalData.

To do that let's add the State and County FP code in the finalData.

In [ ]:
# creating the column in finalData
finalData['STATEFP'] = 0
finalData['COUNTYFP'] = 0
for index, geoid in tqdm(finalData.geoid.items()):
    finalData['STATEFP'][index] = mainData[mainData['geoid'] == geoid]['statefp'].unique()[0]
    finalData['COUNTYFP'][index] = mainData[mainData['geoid'] == geoid]['countyfp'].unique()[0]

In [ ]:
# Adding total population in finalData
finalData['totalPopulation'] = 0
for index in tqdm(range(len(finalData))):
    tempValue = censusData[(censusData['STATE'] == finalData['STATEFP'][index]) &
                          (censusData['COUNTY'] == finalData['COUNTYFP'][index])]
    finalData['totalPopulation'][index] = tempValue[tempValue['YEAR'] == 11]['TOT_POP'].values[0]
    
finalData.head()

Rearranging the column position. 

In [ ]:
finalData = finalData[['STATEFP', 'COUNTYFP', 'geoid', 'CustomerSatisfactionAvgReviewRating', 'CustomerSatisfactionAvgStarRating', 'totalPopulation']]
finalData.head()

Now let's add <b>lodging_inventory_bucket</b> into the finalData. But as can be seen below that there is a big difference in the sum value of lodging_inventory_bucket for vacational and non vacational rentals. So we should create two variables out of it instead of taking their average.

In [ ]:
print('non vacational rental:',mainData[mainData.is_vacation_rental == 0]['lodging_inventory_bucket'].sum())
print('vacational rental:',mainData[mainData.is_vacation_rental == 1]['lodging_inventory_bucket'].sum())

In [ ]:
# populating data of lodging Inventory bucket for vacation rental and non vacation rental in two new columns
finalData['lodgingInventoryBucketNonVacationalRental'] = 0.0
finalData['lodgingInventoryBucketVacationalRental'] = 0.0
for index, value in tqdm(finalData['geoid'].items()):
    finalData.lodgingInventoryBucketNonVacationalRental[index] = mainData[(mainData.months == '2018-10,2018-11') & 
                                                                          (mainData['geoid'] == value ) & 
                                                                          (mainData['is_vacation_rental'] == 0)]['lodging_inventory_bucket'].values[0]
    finalData.lodgingInventoryBucketVacationalRental[index] = mainData[(mainData.months == '2018-10,2018-11') & 
                                                                          (mainData['geoid'] == value ) & 
                                                                          (mainData['is_vacation_rental'] == 1)]['lodging_inventory_bucket'].values[0]
finalData.head()

Some places can have more tourism from other place but we are more interested in number of tourist coming to the county with proportion to the total population. This will give us better understanding of social impact of tourism in counties.

In [ ]:
finalData['lIBNonVRRatioToPopulation'] = finalData['lodgingInventoryBucketNonVacationalRental'] / finalData['totalPopulation']
finalData['lIBVRRatioToPopulation'] = finalData['lodgingInventoryBucketVacationalRental'] / finalData['totalPopulation']

finalData.head()

Once divided by the population. The values become really small. Lets scale them back to a scale from 0 to 100.

In [ ]:
finalData['lIBNonVRRatioToPopulation'] = (finalData['lIBNonVRRatioToPopulation'] / max(finalData['lIBNonVRRatioToPopulation'])) * 100
finalData['lIBVRRatioToPopulation'] = (finalData['lIBVRRatioToPopulation'] / max(finalData['lIBVRRatioToPopulation'])) * 100
finalData.head()

Cool! fun fact, The county with highest value for <b>lIBNonVRRatioToPopulation</b> and <b>lIBVRRatioToPopulation</b> is San Juan County, Colorado. Which has a population of 613 and looks like a very touristic place. 

In [ ]:
print(((finalData['lIBNonVRRatioToPopulation'] / max(finalData['lIBNonVRRatioToPopulation'])) * 100).nlargest(5))
print(censusData[(censusData.STATE == (finalData.loc[300][0])) & (censusData.COUNTY == (finalData.loc[300][1]))].iloc[0]['STNAME'])
print(censusData[(censusData.STATE == (finalData.loc[300][0])) & (censusData.COUNTY == (finalData.loc[300][1]))].iloc[0]['CTYNAME'])

# picture of San Juan county
# ref: https://www.uncovercolorado.com/wp-content/uploads/2018/12/Downtown-Silverton-San-Juan-County-Colorado-1280x640-1094x547.jpg
from IPython.display import Image
Image("../../data/raw/Images/San-Juan-County-Colorado.jpg")

In [ ]:
mainData.head()

A very similar process can be applied on <b>air_inbound_popularity_bucket</b>. Although 87.1% of the values are 0 for this field. We take a mean for values vacational rental and non vacational rental. 

In [ ]:
# populating data of air inbound bucket
finalData['airInboundPopularityBucket'] = 0.0
for index, value in tqdm(finalData['geoid'].items()):
    finalData.airInboundPopularityBucket[index] = mainData[(mainData.months == '2018-10,2018-11') & 
                                                           (mainData['geoid'] == value ) ]['air_inbound_popularity_bucket'].values.mean()

finalData.head()

In [ ]:
# taking a ratio to the population
finalData['aIPBRatioToPopulation'] = finalData['airInboundPopularityBucket'] / finalData['totalPopulation']
# and scaling back to values between 0 and 100
finalData['aIPBRatioToPopulation'] = (finalData['aIPBRatioToPopulation'] / max(finalData['aIPBRatioToPopulation'])) * 100
finalData.head()

We can use some other dataset as well apart from one give officially. We can get the idea of tourism in the county by checking the number of people who are employed in Arts, Entertainment, recreation, accommodation and food services. We can use the Economics Census of 2018 as dataset.

In [ ]:
economicsDataset = pd.read_csv("../../data/raw/EconomyCensus2018/ACSDP1Y2018.DP03_data_with_overlays_2020-01-01T183501.csv")
economicsDataset.head()

Some values of the targeted fields are 'N', let's replace that with 0.

In [ ]:
for index,geoid in tqdm(economicsDataset['GEO_ID'].items()):
    if(economicsDataset[economicsDataset['GEO_ID'] == geoid]['DP03_0043E'].values[0] == 'N'):
        economicsDataset.loc[index,'DP03_0043E'] = '0'
    if(economicsDataset[economicsDataset['GEO_ID'] == geoid]['DP03_0043PE'].values[0] == 'N'):
        economicsDataset.loc[index,'DP03_0043PE'] = '0'

In [ ]:
import math

# populating data people emplyed in tourism related industry
finalData['AERAFEmployement'] = 0.0
finalData['AERAFEmployementPercent'] = 0.0
for index, value in tqdm(finalData['geoid'].items()):
    if((int(math.log10(value))+1) == 4):
        if(economicsDataset['GEO_ID'].str.contains('0500000US0' + str(value)).any()):
            finalData.AERAFEmployement[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US0' + str(value))]['DP03_0043E'].values[0])
            finalData.AERAFEmployementPercent[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US0' + str(value))]['DP03_0043PE'].values[0])
    elif((int(math.log10(value))+1) == 5):
        if(economicsDataset['GEO_ID'].str.contains('0500000US' + str(value)).any()):
            finalData.AERAFEmployement[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US' + str(value))]['DP03_0043E'].values[0])
            finalData.AERAFEmployementPercent[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US' + str(value))]['DP03_0043PE'].values[0])
        
finalData.head()

73% of the values are missing. We can come back to fixing these missing values later. for now, we can  mark a check point here. 

In [ ]:
finalData.to_csv(r'../../data/processed/tempDataFrameLandmark/finalData2ndCheckpoint.csv')  

## Economic impact
### Vishal 

Tourism affect the economy of the region and same holds for vice versa. In this section of the notebook we will try to analyze economic factor that impact community and hence tourism. 

<h4>Income, Employment and Pay Gap</h4>
Starting with income of people in the region. Better average income of the society indicate more prosperity. let's start playing with the data we have related with the income. 

* Data used: SELECTED ECONOMIC CHARACTERISTICS, 2018: ACS 1-Year Estimates Data profiles
* url: https://data.census.gov/cedsci/table?q=United%20States&g=0100000US,.050000&table=DP03&tid=ACSDP1Y2018.DP03&hidePreview=true&vintage=2018&lastDisplayedRow=144

The data is stored in raw folder as a CSV file.

Let's start with looking at the median household income. It will give a good insight economic state of the county. 

In [ ]:
# populating data of median household income in county
finalData['medianHouseHoldIncome'] = 0.0
for index, value in tqdm(finalData['geoid'].items()):
    if((int(math.log10(value))+1) == 4):
        if(economicsDataset['GEO_ID'].str.contains('0500000US0' + str(value)).any()):
            finalData.medianHouseHoldIncome[index] = int(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US0' + str(value))]['DP03_0062E'].values[0])
    elif((int(math.log10(value))+1) == 5):    
        if(economicsDataset['GEO_ID'].str.contains('0500000US' + str(value)).any()):
            finalData.medianHouseHoldIncome[index] = int(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US' + str(value))]['DP03_0062E'].values[0])

finalData.head()            

Another indicator that we can consider is percent of unemployed people in the county.

In [ ]:
# populating data of unemployement percent in the population
finalData['unEmployementRate'] = 0.0
for index, value in tqdm(finalData['geoid'].items()):
    if((int(math.log10(value))+1) == 4):
        if(economicsDataset['GEO_ID'].str.contains('0500000US0' + str(value)).any()):
            finalData.unEmployementRate[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US0' + str(value))]['DP03_0009PE'].values[0])
    elif((int(math.log10(value))+1) == 5):    
        if(economicsDataset['GEO_ID'].str.contains('0500000US' + str(value)).any()):
            finalData.unEmployementRate[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US' + str(value))]['DP03_0009PE'].values[0])

finalData.head()            

Under economics of the county we are interested in seeing the percent of people in that county under poverty level.

In [ ]:
# populating data of unemployement percent in the population
finalData['povertyRate'] = 0.0
for index, value in tqdm(finalData['geoid'].items()):
    if((int(math.log10(value))+1) == 4):
        if(economicsDataset['GEO_ID'].str.contains('0500000US0' + str(value)).any()):
            finalData.povertyRate[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US0' + str(value))]['DP03_0128PE'].values[0])
    elif((int(math.log10(value))+1) == 5):    
        if(economicsDataset['GEO_ID'].str.contains('0500000US' + str(value)).any()):
            finalData.povertyRate[index] = float(economicsDataset[economicsDataset['GEO_ID'] == ('0500000US' + str(value))]['DP03_0128PE'].values[0])

finalData.head() 

Finally, we are interested in looking at the gender pay-gap. 

## Cultural impact

With the official census data, we can start by looking at the racial diversity of the county. 

In [ ]:
# Ratio of minority population to total population
finalData['minorityPopulationRatio'] = 0.0
for index in tqdm(range(len(finalData))):
    tempValue = censusData[(censusData['STATE'] == finalData['STATEFP'][index]) &
                          (censusData['COUNTY'] == finalData['COUNTYFP'][index])]
    minorityPopulationSum = tempValue.loc[tempValue['YEAR'] == 11, ['BA_MALE','BA_FEMALE','IA_MALE','IA_FEMALE','AA_MALE','AA_FEMALE','NA_MALE','NA_FEMALE']].values[0].sum()
    finalData['minorityPopulationRatio'][index] = minorityPopulationSum / (tempValue[tempValue['YEAR'] == 11]['TOT_POP'].values[0])
    
finalData.head()

Let's rescale the minority PopulationRatio to 0-100.

In [ ]:
finalData['minorityPopulationRatio'] = (finalData['minorityPopulationRatio'] / max(finalData['minorityPopulationRatio'])) * 100
finalData.head()